# API da Gmail

## O que é uma API

**API** é a sigla para Application Programming Interface (Interface de Programação de Aplicações). Em termos simples, é um conjunto de regras e ferramentas que permite que diferentes softwares se comuniquem entre si.

A API:

- Recebe seu pedido

- Verifica se você tem permissão (autenticação)

- Busca os dados no servidor do Gmail

- Retorna a resposta em um formato padronizado (geralmente JSON)

Com o uso de APIs todas as interações seguem regras claras, o banco de de dados nunca é acessado direto do servidor e isso garante segurança dos usuários, você recebe somente as informações que pediu e não precisa saber como o Gmail é implementado por dentro.

Os principais componente de uma API são:

- Endpoint: é uma URL especifica para cada recurso, por exemplo, "https://gmail.googleapis.com/gmail/v1/users/me/messages".

- [Método](https://developers.google.com/workspace/gmail/api/reference/rest/v1/users.messages/get?hl=pt-br): Ação que você quer executar, por exemplo, GET (listar), POST (criar) ou DELETE (remover).

- Parâmetros: São as informações adicionais da requisição, por exemplo, maxResults=32.

- Autenticação: Prova quem você é, por exemplo, Token OAuth 2.0.

- Resposta: Dados retornados pela API, geralmente em formato JSON.

O fluxo básico envolve :

1. Criar um projeto no Google Cloud Console

2. Ativar a Gmail API

3. Configurar a tela de consentimento OAuth

4. Criar credenciais (cliente OAuth 2.0)

5. Implementar o fluxo de autenticação no seu app

Nesse texto será mostrado todas essas componentes e mostrado como se trabalha com a API do Gmail.


## [Preparação do Ambiente](https://developers.google.com/workspace/gmail/api/quickstart/python?hl=pt-br)


Para iniciar o fluxo de autenticação e de autorização é necessário:

- Python 3.10.7 ou mais recente
- A ferramenta de gerenciamento de pacotes pip
- Uma Conta do Google com o Gmail ativado.
- Um projeto do Google Cloud.

### 1. Como criar um projeto no google cloud

Esse projeto organizará as APIs que você usa, as credenciais de acesso e o faturamento.


1. Com seu gmail logado, acesse [Google Cloud](https://console.cloud.google.com/)

2. No topo da página, ao lado direito do logotipo do Google Cloud, clique no seletor de projetos (que pode estar escrito "Selecione um projeto" ou mostrando o nome de um projeto atual) . No menu que abrir, clique em "Novo Projeto".

3. Configure os Detalhes do Projeto:

- Uma tela chamada "Novo Projeto" será exibida. Preencha os campos :

  - Nome do projeto: Escolha um nome descritivo (ex: "Meu Projeto Gmail API"). É com esse nome que você o identificará no console.

  - Local (Organização): Se sua conta estiver associada a uma organização, você poderá escolher uma pasta. Para uso pessoal, a opção "Sem organização" é a mais comum.

O ID do Projeto será gerado automaticamente a partir do nome, mas você pode clicar em "Editar" para personalizá-lo. Atenção: este ID é único e não poderá ser alterado depois .

4. Clique no botão "Criar". 

5. Selecione o Novo Projeto: Após a criação, uma notificação aparecerá. Clique em "Selecionar projeto" para ser redirecionado para o painel do seu novo projeto . Verifique se o nome do seu projeto está visível no seletor de projetos no topo da página (Ao lado direito do logotipo do google cloud).

### 2. Ativação da API

É necessário ativar a API para que o script Python consiga se conectar com o Gmail.



Dentro do seu [projeto](https://console.cloud.google.com/apis/enableflow?apiid=gmail.googleapis.com&hl=pt-br), vá em ** "APIs e Serviços" > "Biblioteca"**

Procure por "Gmail API" e clique em "Ativar"

### 3. Consentimento OAuth

É necessário ativar a tela de consentimento OAuth para que a API do Gmail de identifique quando necessário.

Na barra lateral esquerda do Google Cloud Platform clique em  *"APIs e Serviços" > "Tela de consentimento OAuth"*.

Escolha "Interno" e preencha as informações básicas como o nome do app, e-mail de suporte,

### 4. Criação das Credenciais

As credenciais identificam seu script para o Google, permitindo que ele solicite autorização do usuário para acessar os dados do Gmail. Sem ele, o fluxo de autenticação OAuth 2.0 não pode ser iniciado, pois o Google não sabe qual aplicativo está pedindo acesso. 

Com credentials.json seu script consegue pedir autorização, gerar o token.json e finalmente acessar a API. É como um documento de identidade do seu aplicativo.

Na mesma tela, "APIs e Serviços", vá em:

**"Credenciais" e clique em "+ Criar Credenciais" > "ID do cliente OAuth"**

Escolha "Aplicativo da Web" e adicione um nome.

Após criar, baixe o arquivo JSON. 

Renomeie esse arquivo para credentials.json e coloque-o na raiz do seu projeto. 

É este arquivo que seu código lê quando chama 

```bash 
InstalledAppFlow.from_client_secrets_file()
```

## 5 .Criação do script Python 

[Fonte](https://developers.google.com/workspace/gmail/api/quickstart/python?hl=pt-br)

Inicie instalando a biblioteca de cliente do Google para o Python.

Digite em seu terminal:

```bash
python3 -m pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

### Script de fluxo de autenticação e autorização

In [1]:
import os.path
from pathlib import Path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64



In [2]:
# O escopo concede ao seu aplicativo acesso somente leitura a todos os recursos da caixa de correio do usuário. 
# Com ele, seu script pode listar e-mails e seus IDs, baixar mensagens e seus metadados, 
# mas não pode enviar ou excluir mensagens.
# Obs: Se modificar os escopos, exclua o arquivo token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"] # Acesso somente leitura à caixa de correio do usuário

"""Mostra o uso básico da API do Gmail."""
  

creds = None

# O arquivo token.json armazena as credenciais de acesso do usuário. 
# Ele é criado automaticamente quando o fluxo de autorização é concluído pela primeira vez.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# Se as credenciais forem inválidas ou expiradas, solicita login do usuário.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)

# Salva as credenciais para a próxima execução
    with open("token.json", "w") as token:
        token.write(creds.to_json())

try:
# Chama a API do Gmail
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
    print(f"An error occurred: {error}")

#### Automatização da autenticação

O script de autenticação e autorização pode ser automatizado da seguinte forma:

In [3]:

# Escopos de acesso/ExtractEmais.py
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

# Definição dos caminhos (ajuste se necessário)

TOKEN = "token.json"
CREDENTIAL = "credential.json"
DIRETORIO_RAIZ = Path.cwd()
PROCESSED_IDS_PATH = DIRETORIO_RAIZ / "data"
PROCESSED_ATTACHMENTS_PATH = PROCESSED_IDS_PATH / "attachments"
PROCESSED_IDS_FILE="processed_ids.json"

def get_gmail_service():
    """Autentica e retorna o serviço da Gmail API."""
    creds = None

    # Carrega token existente, se houver
    if os.path.exists(TOKEN):
        creds = Credentials.from_authorized_user_file(TOKEN, SCOPES)

    # Se não há credenciais válidas, faz login
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)

        # Salva o token para a próxima execução
        with open(TOKEN, "w") as token:
            token.write(creds.to_json())

    try:
        # Testa a autenticação listando as labels
        service = build("gmail", "v1", credentials=creds)
        service.users().labels().list(userId="me").execute()
        print("Autenticação bem-sucedida!")
        return service
    except HttpError as error:
        print(f"Falha ao autenticar o serviço do Gmail: {error}")
        return None

A partir de agora, para autenticar o serviço, basta todar o comando abaixo.

In [4]:
get_gmail_service()

Autenticação bem-sucedida!


### [Listar Mensagens](https://developers.google.com/workspace/gmail/api/guides/list-messages?hl=pt-br)

O método list tem como principal objetivo retornar uma lista de IDs de mensagens que estão na caixa de correio do usuário.

**users.messages.list** é compatível com vários parâmetros de consulta para filtrar as mensagens:

- **maxResults**: número máximo de mensagens a serem retornadas (o padrão é 100, e o máximo é 500).
- **pageToken**: token para recuperar uma página específica de resultados.
- **q**: string de consulta para filtrar mensagens, como from:someuser@example.com is:unread".
- **labelIds**: retorna apenas mensagens com rótulos que correspondem a todos os IDs de rótulo especificados.
- **includeSpamTrash**: inclua mensagens de SPAM e TRASH nos resultados.

Quando você lista os emails de uma caixa de entrada, usando o método list, você recebe apenas metadados básicos e os IDs. Para obter o conteúdo completo de um email (assunto, corpo, anexos, etc...), você precisa chamar o método get para cada ID de interesse. 

Saiba mais sobre:

* O método get [aqui](https://developers.google.com/workspace/gmail/api/reference/rest/v1/users.messages/list?hl=pt-br)

* Como pesquisar mensagens [aqui](https://developers.google.com/workspace/gmail/api/guides/filtering?hl=pt-br)

* Como criar filtro de e-mails [aqui](https://support.google.com/mail/answer/7190?hl=pt-br)

* O método list [aqui](https://developers.google.com/workspace/gmail/api/reference/rest/v1/users.messages/get?hl=pt-br)

In [5]:
 # Lista os rótulos da caixa de correio do usuário.    
results_labels = service.users().labels().list(userId="me").execute()

print(f'Resultados: {results_labels}')

# Obtém a lista de rótulos. Se não houver rótulos, a lista será vazia.
labels = results_labels.get("labels", [])
if labels:
    print("Labels:")

    for label in labels:
        print(label["name"])
else:
    print("No labels found.")    

Resultados: {'labels': [{'id': 'CHAT', 'name': 'CHAT', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelShow', 'type': 'system'}, {'id': 'SENT', 'name': 'SENT', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelShow', 'type': 'system'}, {'id': 'INBOX', 'name': 'INBOX', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelShow', 'type': 'system'}, {'id': 'IMPORTANT', 'name': 'IMPORTANT', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelShow', 'type': 'system'}, {'id': 'TRASH', 'name': 'TRASH', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelHide', 'type': 'system'}, {'id': 'DRAFT', 'name': 'DRAFT', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelShow', 'type': 'system'}, {'id': 'SPAM', 'name': 'SPAM', 'messageListVisibility': 'hide', 'labelListVisibility': 'labelHide', 'type': 'system'}, {'id': 'CATEGORY_FORUMS', 'name': 'CATEGORY_FORUMS', 'type': 'system'}, {'id': 'CATEGORY_UPDATES', 'name': 'CATEGORY_UPDATES', 

In [8]:
results_messages = service.users().messages().list(userId="me").execute()
mensagens = results_messages.get("messages", [])

# print('Resultados')
# if results_messages:
#     for message in mensagens:
#         print(message["id"])
# else:
#     print("No messages found.")

```bash
results_messages = service.users().messages().list(userId="me").execute()
mensagens = results_messages.get("messages", [])

print('Resultados')
if results_messages:
    for message in mensagens:
        print(message["id"])
else:
    print("No messages found.")
```

Output:

Resultados
19c8a6ed637e6bcf
19c8a50e83be0949
19c8a6b336e6331d
19c8925a2630ab71
19c882c8f6c09c78
19c87e51cf29d446
19c87cc42c9a343f


* *page_token = results.get('nextPageToken')*

page_token é o id da página de e-mail.
A variável page_token controla o loop. Ela começa como None e é atualizada a cada iteração com o nextPageToken vindo da API

In [9]:
page_token = None
results=service.users().messages().list(
                userId='me',
                pageToken=page_token #token para controle de paginação
            ).execute()

page_token = results.get('nextPageToken')
print(f'Próxima página: {page_token}')

Próxima página: 10477622702421839269


O script abaixo mostra o token das primeiras 10 páginas do e-mail.

In [10]:
page_token = None
contador = 0
maxResults = 10 # Defina o número máximo de páginas que deseja percorrer
while contador<maxResults:  
    results=service.users().messages().list(
                    userId='me',
                    maxResults=maxResults,
                    pageToken=page_token #token para controle de paginação
                ).execute()
    print(f'Próxima página: {page_token}')
    page_token = results.get('nextPageToken')
    contador+=1


Próxima página: None
Próxima página: 17805674181809609267
Próxima página: 06778736563025220617
Próxima página: 11167327824144860516
Próxima página: 11620997087831798058
Próxima página: 15967831412322366290
Próxima página: 03650172437270654438
Próxima página: 01368546908185192048
Próxima página: 13215718540133858177
Próxima página: 17137005678544621845


#### Automatização da seleção dos Ids dos de e-mails filtrados

In [11]:
def fetch_email_ids(service, days_back=None, max_results=None, has_attachments=None, from_email=None):
    """
    Busca IDs de e-mails da caixa de entrada com filtros opcionais.
    
    Args:
        service: Serviço autenticado da Gmail API.
        days_back: Número de dias para trás (se None, busca todos).
        max_results: Número máximo de IDs a retornar (se None, busca todos via paginação).
        has_attachments: Se True, filtra apenas e-mails com anexos.
    
    Returns:
        Lista de IDs (strings).
    """
    
    # Pinta os filtros aplicados
    print(f'''
          Filtros aplicados: 
          - days_back={days_back} dias, 
          - max_results={max_results}, 
          - has_attachments={has_attachments}, 
          - from_email={from_email}
          ------------------------''')
    
    # Verifica se o serviço foi autenticado corretamente
    if service is None:
        print("Serviço não disponível.")
        return []

    all_ids = [] # Lista para armazenar os IDs coletados
    page_token = None # Token para controle de paginação, inicia como None para a primeira página
    query_parts = ["in:inbox"] # Começa a query buscando apenas na caixa de entrada

    # Aplica os filtros à query
    # Se os comandos abaixo forem fornecidos, adiciona o respectivo filtro em query_parts
    if days_back is not None:
        query_parts.append(f"newer_than:{days_back}d")

    if has_attachments=='True':
        query_parts.append("has:attachment")

    if has_attachments=='False':
        query_parts.append("-has:attachment")
    
    if from_email is not None:
        query_parts.append(f"from:{from_email}")

    # Une os filtros em uma única string de consulta
    query = " ".join(query_parts)

    # Define o tamanho da página (máx 500, ou o limite solicitado se menor)
    page_size = 500
    if max_results is not None and max_results < page_size:
        page_size = max_results

    # Loop para buscar páginas de resultados até atingir o máximo desejado ou não haver mais páginas
    while True:
        try:
            # Faz a chamada à API para listar mensagens com os filtros e paginação
            results = service.users().messages().list(
                userId='me',
                q=query if query else None,
                maxResults=page_size,
                pageToken=page_token #token para controle de paginação
            ).execute()
        
        # Se ocorrer um erro na chamada da API, imprime o erro e interrompe o loop    
        except HttpError as error:
            print(f"Erro na chamada da API: {error}")
            break
        
        messages = results.get('messages', [])
        if messages: # Se houver mensagens nesta página, extrai os IDs e adiciona à lista total
            page_ids = [msg['id'] for msg in messages]
            all_ids.extend(page_ids)

        # Se já atingimos o máximo desejado, pára
        if max_results is not None and len(all_ids) >= max_results:
            all_ids = all_ids[:max_results]
            break
        
        if len(all_ids)==0:
            print("Nenhum e-mail encontrado com os filtros aplicados.")
            break

        # Verifica se há mais páginas
        page_token = results.get('nextPageToken')
        if not page_token:
            break

        # Ajusta o page_size para não exceder o restante necessário
        if max_results is not None:
            remaining = max_results - len(all_ids)
            page_size = min(500, remaining)
    return all_ids

In [12]:
# ID dos primeiros 5 e-mails da caixa de entrada    
fetch_email_ids(service, max_results=5)


          Filtros aplicados: 
          - days_back=None dias, 
          - max_results=5, 
          - has_attachments=None, 
          - from_email=None
          ------------------------


['19c8a6bd637e6bcf',
 '19c8a30e83be0949',
 '19c8a0b336e6331d',
 '19c8921a2630ab71',
 '19c882c9f6c09c78']

In [13]:
#  ID dos primeiros 10 e-mails com anexos                                                                                               
fetch_email_ids(service, max_results=2000, has_attachments='True')                                                        


          Filtros aplicados: 
          - days_back=None dias, 
          - max_results=2000, 
          - has_attachments=True, 
          - from_email=None
          ------------------------


['19c85900103d3820',
 '19c83648b6d429ac',
 '19c7fa9313ec362d',
 '19c7a3049557d32f',
 '19c77bd6684a8642',
 '19c7755fb7457e52',
 '19c71ac5f70a575e',
 '19c5da9fee06d27f',
 '19c43f43358cfb66',
 '19c42cc810f38566',
 '19c3337d3fef4b80',
 '19c2f50ceea27d7d',
 '19c2a4dd90de2ea7',
 '19c24ba39cf78f74',
 '19c2488a919e3b16',
 '19c244a44096609f',
 '19c22fe16e3b86c9',
 '19c19f08b08608cc',
 '19c097beffed5dde',
 '19c046861f79b0fe',
 '19bffc70d031e265',
 '19bfadeda72032bc',
 '19bf33271a2c1f79',
 '19be20d52a0c4e8c',
 '19be1893dfb99908',
 '19bdd65815b5b2a5',
 '19bcd41339a67e47',
 '19bc8484acb42975',
 '19bc2d6002c6ca54',
 '19bc27f3b3fa3f98',
 '19baa1510b772a7c',
 '19ba02be31d23724',
 '19b93cb13f250a30',
 '19b93601d5b274f6',
 '19b8aba641805d6d',
 '19b8aa94429ac6a6',
 '19b83aa0a0cbd562',
 '19b748dcadc1a2e9',
 '19b71a9aa44a210b',
 '19b6309e69a65631',
 '19b60e6080f3b816',
 '19b5d3508407cb9c',
 '19b53918931fe501',
 '19b46524a251b51f',
 '19b42d42dd43c68a',
 '19b28937acba6ad3',
 '19b2204647435e21',
 '19b21c81ec5

In [14]:
# ID dos primeiros 10 e-mails com anexos nos últimos 3 dias. 
# Observe que não há 10 e-mails com anexos nos últimos 3 dias, então o resultado será menor que 10.
fetch_email_ids(service, max_results=10, has_attachments='True', days_back=3) 


          Filtros aplicados: 
          - days_back=3 dias, 
          - max_results=10, 
          - has_attachments=True, 
          - from_email=None
          ------------------------


['19c85900103d3820', '19c83648b6d429ac', '19c7fa9313ec362d']

In [15]:
# Mostra os IDs dos primeiros 2 e-mails com anexos do remetente contadigital@mhnet.com.br
fetch_email_ids(service, max_results=2, has_attachments='True', from_email='contadigital@mhnet.com.br')


          Filtros aplicados: 
          - days_back=None dias, 
          - max_results=2, 
          - has_attachments=True, 
          - from_email=contadigital@mhnet.com.br
          ------------------------


['19bffc70d031e265', '19b93601d5b274f6']

In [16]:
# Mostra os IDs dos primeiros 2 e-mails com anexos do remetente blabla@email.com.br
fetch_email_ids(service, max_results=2, has_attachments='True', from_email='blabla@email.com.br')


          Filtros aplicados: 
          - days_back=None dias, 
          - max_results=2, 
          - has_attachments=True, 
          - from_email=blabla@email.com.br
          ------------------------
Nenhum e-mail encontrado com os filtros aplicados.


[]

A próxima etapa consiste em automatizar o salvamento dos ids processados

In [17]:
def save_processed_ids(processed_ids):
    """Salva o conjunto de IDs processados no arquivo JSON."""
    
    # Garante que o diretório existe
    PROCESSED_IDS_PATH.mkdir(parents=True, exist_ok=True)
    
    with open(os.path.join(PROCESSED_IDS_PATH, PROCESSED_IDS_FILE), 'w') as f:
        json.dump(list(processed_ids), f, indent=2)
    
    print(f"IDs salvos em {PROCESSED_IDS_PATH / PROCESSED_IDS_FILE}")


#### Downloads dos anexos

In [18]:
message_id = '19b93601d5b274f6'  # Substitua pelo ID do e-mail que deseja baixar
# gera um dict em que as chaves são: 'id', 'threadId', 'labelIds', 'snippet', 'historyId', 'internalDate', 'payload', 'sizeEstimate', 'raw' (nem todas as mensagens terão a chave 'raw')
message = service.users().messages().get(
            userId='me',
            id=message_id,
            format='full'
        ).execute()
message.keys()

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])

In [19]:
# Recebe o conteúdo da mensagem em formato de dict em que as chaves são: 
# 'partId', 'mimeType', 'filename', 'headers', 'body', 'parts' (nem todas as mensagens terão a chave 'parts')
message.get('payload', {}).keys()

dict_keys(['partId', 'mimeType', 'filename', 'headers', 'body', 'parts'])

In [20]:
# Recebe a lista de partes da mensagem (se houver). 
# Cada parte é um dict com as chaves mencionadas acima.
message.get('payload', {}).get('parts', [])

[{'partId': '0',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Content-Type',
    'value': 'multipart/alternative; boundary="=-NkaLj6FRexJ1u4E5Bq8Ejg=="'}],
  'body': {'size': 0},
  'parts': [{'partId': '0.0',
    'mimeType': 'text/plain',
    'filename': '',
    'headers': [{'name': 'Content-Type',
      'value': 'text/plain; charset=utf-8'}],
    'body': {'size': 192,
     'data': 'U3VhIGZhdHVyYSBNaG5ldCBlc3TDoSBkaXNwb27DrXZlbCENCg0Kw4kgc8OzIGNsaWNhciBlbSAiUGFnYXIgRmF0dXJhIiBhcXVpIGVtYmFpeG8g8J-RhyBlIHZvY8OqIHBvZGVyw6EgZXNjb2xoZXIgYSBmb3JtYSBkZSBwYWdhbWVudG8gcXVlIHByZWZlcmlyOiA8Yj4gQ2FydMOjbyBkZSBDcsOpZGl0bywgUGl4IG91IGJvbGV0by48L2I-'}},
   {'partId': '0.1',
    'mimeType': 'text/html',
    'filename': '',
    'headers': [{'name': 'Content-Type', 'value': 'text/html; charset=utf-8'},
     {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}],
    'body': {'size': 23080,
     'data': 'PCFET0NUWVBFIGh0bWw-DQo8aHRtbCBsYW5nPSJlbiIgeG1sbnM9

In [ ]:
# Para cada parte do Id, verifica se há um anexo e, se houver, 
# baixa o anexo usando o ID da mensagem e o ID do anexo.
attachment=service.users().messages().attachments().get(
                    userId='me',
                    messageId=message_id,
                    id='ANGjdJ-aSQjOp7FaOdEPfVbPiHuuX_Y7VDYrc3eCiKDi7ZIFJtG3_UywVOdoz8v0Y-niCHHkHWisVBoF5o6XwHWwxndI2EIXgCzY4SAWo2jhT2C-Y0-ZH1nmQu_cEG-v7ph2GxyLKSCpDXTZUXgef0VagpPsqjnfxucijfibyIrDFXIvI0zd639Dx2SpUi9g8BQugtQodEJK1q_MIUjQC7p5bRLTgY7xLSarwabdEXb1KyniJeio6L-UxpSG4DpKsW91_mR9PS8cGr_M5HVIMn1b8VEIQezLEL622hsRiLD7jiAmNxLg_v9aoQ8nXPLom2LbH0U8q05k_ZB6DntV5P2Fib0OYzicinofkH_B6zsAi5D9ybjLOig_OloPJZf6AiH92i82zkIaMkwXUGoJooztu4QM-qEEMcetA'
                ).execute()

In [ ]:
# O conteúdo do anexo é codificado em base64, 
# então precisamos decodificá-lo para obter os bytes originais do arquivo.
base64.urlsafe_b64decode(attachment['data'])

In [23]:
downloaded_files = []
parts = message.get('payload', {}).get('parts', [])

In [24]:
for part in parts:

    # Verifica se há um um anexo
    if 'filename' in part and part['filename']: 
        filename = part['filename']
        
        # Constrói caminho seguro para o arquivo, 
        # trocando caracteres inválidos por sublinhados
        safe_filename = filename.replace('/', '_').replace('\\', '_')
        filepath = os.path.join(PROCESSED_ATTACHMENTS_PATH, safe_filename)
        print(f"Arquivo seguro: {filepath}")
        
        # Evita sobrescrever arquivos com nomes iguais
        if os.path.exists(filepath):
            base, ext = os.path.splitext(safe_filename) # Separa o nome do arquivo e a extensão
            counter = 1
            while os.path.exists(filepath):
                new_filename = f"{base}_{counter}{ext}"
                filepath = os.path.join(PROCESSED_ATTACHMENTS_PATH, new_filename)
                counter += 1
                
        # Obtém o attachmentId
        if 'body' in part and 'attachmentId' in part['body']:
            # Pega o ID do anexo para baixar o arquivo posteriormente
            attachment_id = part['body']['attachmentId']
            
            try:
                # 4. Baixa o anexo usando attachments().get()
                attachment = service.users().messages().attachments().get(
                    userId='me',
                    messageId=message_id,
                    id=attachment_id
                ).execute()
                
                # 5. Decodifica os dados baixados do anexo(base64url) 
                file_data = base64.urlsafe_b64decode(attachment['data'])
                
                # 6. Salva o arquivo
                with open(filepath, 'wb') as f:
                    f.write(file_data)
                
                print(f"Anexo salvo: {filepath}")
                downloaded_files.append(filepath)
            except HttpError as e:
                print(f"  Erro ao baixar anexo {filename}: {e}")
            except Exception as e:
                print(f"  Erro inesperado ao baixar {filename}: {e}")

Arquivo seguro: /home/priscila/Documentos/estudos/portfolio/data/attachments/Fatura_MhnetTelecom.pdf
Anexo salvo: /home/priscila/Documentos/estudos/portfolio/data/attachments/Fatura_MhnetTelecom_1.pdf


In [25]:
# Caminho para salvar os IDs filtrados, cria a pasta "filtered_ids" se não existir.
caminho = PROCESSED_IDS_PATH / "filtered_ids"
print("Criando em:", caminho)
caminho.mkdir(parents=True, exist_ok=True)

Criando em: /home/priscila/Documentos/estudos/portfolio/data/filtered_ids


In [26]:
def process_parts(parts, parent_path=""):

    if not parts:
        return

    for part in parts:
        # Verifica se esta parte tem filename (é um anexo)
        if 'filename' in part and part['filename']:
            filename = part['filename']
            
            # Constrói caminho seguro para o arquivo
            safe_filename = filename.replace('/', '_').replace('\\', '_')
            filepath = os.path.join(download_dir, safe_filename)
            
            # Evita sobrescrever arquivos com nomes iguais
            if os.path.exists(filepath):
                base, ext = os.path.splitext(safe_filename)
                counter = 1
                while os.path.exists(filepath):
                    new_filename = f"{base}_{counter}{ext}"
                    filepath = os.path.join(download_dir, new_filename)
                    counter += 1
            
            # Obtém o attachmentId
            if 'body' in part and 'attachmentId' in part['body']:
                attachment_id = part['body']['attachmentId']
                
                try:
                    # 4. Baixa o anexo usando attachments().get()
                    attachment = service.users().messages().attachments().get(
                        userId='me',
                        messageId=message_id,
                        id=attachment_id
                    ).execute()
                    
                    # 5. Decodifica os dados (base64url) 
                    file_data = base64.urlsafe_b64decode(attachment['data'])
                    
                    # 6. Salva o arquivo
                    with open(filepath, 'wb') as f:
                        f.write(file_data)
                    
                    print(f"  ✅ Anexo salvo: {filepath}")
                    downloaded_files.append(filepath)
                    
                except HttpError as e:
                    print(f"  Erro ao baixar anexo {filename}: {e}")
                except Exception as e:
                    print(f"  Erro inesperado ao baixar {filename}: {e}")
        
        # Processa recursivamente subpartes (mensagens com partes aninhadas)
        if 'parts' in part:
            process_parts(part['parts'], parent_path)

In [27]:
message = service.users().messages().get(
            userId='me',
            id=message_id,
            format='full'
        ).execute()


A função que será definida a seguir download_attachment, baixa todos os anexos de uma mensagem específica do Gmail e salva no diretório *attachments*

In [28]:
def download_attachments(service, message_id, download_dir="attachments"):
    """
    Baixa todos os anexos de uma mensagem específica do Gmail.
    
    Args:
        service: Serviço autenticado da Gmail API.
        message_id: ID da mensagem (string).
        download_dir: Diretório onde salvar os anexos (padrão: "attachments").
    
    Returns:
        Lista de caminhos dos arquivos baixados (strings).
    """
    downloaded_files = []
    
    try:
        # 1. Obtém a mensagem completa com formato 'full' para ter acesso aos parts
        message = service.users().messages().get(
            userId='me',
            id=message_id,
            format='full'
        ).execute()
        
        # 2. Cria o diretório de download se não existir
        Path(download_dir).mkdir(parents=True, exist_ok=True)

        # 3. Processa as partes da mensagem para encontrar anexos
        process_parts(parts, parent_path="")  

        # Inicia o processamento a partir do payload principal
        # payload é a parte da mensagem que contém todo o conteúdo útil do e-mail
        if 'payload' in message:
            payload = message['payload']
            
            # Se o payload principal já for um anexo
            if 'filename' in payload and payload['filename']:
                process_parts([payload])
            # Se tiver partes, processa-as
            elif 'parts' in payload:
                process_parts(payload['parts'])
        
        return downloaded_files
    
    except HttpError as error:
        print(f"Erro na API ao processar mensagem {message_id}: {error}")
        return []
    except Exception as error:
        print(f"Erro inesperado: {error}")
        return []
        

In [29]:
def download_attachments(service, message_id, download_dir="attachments"):
    """
    Baixa todos os anexos de uma mensagem específica do Gmail.
    
    Args:
        service: Serviço autenticado da Gmail API.
        message_id: ID da mensagem (string).
        download_dir: Diretório onde salvar os anexos (padrão: "attachments").
    
    Returns:
        Lista de caminhos dos arquivos baixados (strings).
    """
    downloaded_files = []
    
    try:
        # 1. Obtém a mensagem completa com formato 'full' para ter acesso aos parts
        message = service.users().messages().get(
            userId='me',
            id=message_id,
            format='full'
        ).execute()
        
        # 2. Cria o diretório de download se não existir
        Path(download_dir).mkdir(parents=True, exist_ok=True)
        
        # 3. Função recursiva para percorrer as partes da mensagem
        def process_parts(parts, parent_path=""): # parts inicialmente é message.get('payload', {}).get('parts', [])
            nonlocal downloaded_files
            
            if not parts:
                return
            
            for part in parts:
                # Verifica se esta parte tem filename (é um anexo)
                if 'filename' in part and part['filename']:
                    filename = part['filename']
                    
                    # Constrói caminho seguro para o arquivo
                    safe_filename = filename.replace('/', '_').replace('\\', '_')
                    filepath = os.path.join(download_dir, safe_filename)
                    
                    # Evita sobrescrever arquivos com nomes iguais
                    if os.path.exists(filepath):
                        base, ext = os.path.splitext(safe_filename)
                        counter = 1
                        while os.path.exists(filepath):
                            new_filename = f"{base}_{counter}{ext}"
                            filepath = os.path.join(download_dir, new_filename)
                            counter += 1
                    
                    # Obtém o attachmentId
                    if 'body' in part and 'attachmentId' in part['body']:
                        attachment_id = part['body']['attachmentId']
                        
                        try:
                            # 4. Baixa o anexo usando attachments().get()
                            attachment = service.users().messages().attachments().get(
                                userId='me',
                                messageId=message_id,
                                id=attachment_id
                            ).execute()
                            
                            # 5. Decodifica os dados (base64url) 
                            file_data = base64.urlsafe_b64decode(attachment['data'])
                            
                            # 6. Salva o arquivo
                            with open(filepath, 'wb') as f:
                                f.write(file_data)
                            
                            print(f"  ✅ Anexo salvo: {filepath}")
                            downloaded_files.append(filepath)
                            
                        except HttpError as e:
                            print(f"  Erro ao baixar anexo {filename}: {e}")
                        except Exception as e:
                            print(f"  Erro inesperado ao baixar {filename}: {e}")
                
                # Processa recursivamente subpartes (mensagens com partes aninhadas)
                if 'parts' in part:
                    process_parts(part['parts'], parent_path)
        
        # Inicia o processamento a partir do payload principal
        if 'payload' in message:
            payload = message['payload']
            
            # Se o payload principal já for um anexo
            if 'filename' in payload and payload['filename']:
                process_parts([payload])
            # Se tiver partes, processa-as
            elif 'parts' in payload:
                process_parts(payload['parts'])
        
        return downloaded_files
        
    except HttpError as error:
        print(f"Erro na API ao processar mensagem {message_id}: {error}")
        return []
    except Exception as error:
        print(f"Erro inesperado: {error}")
        return []

In [30]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import json

In [35]:
def main():
   
    parser = ArgumentParser(
        description='Extrai e-mails da caixa de entrada do Gmail e os salva em um arquivo JSON.',
        formatter_class=ArgumentDefaultsHelpFormatter
    )

    parser.add_argument(
        '--max_results',
        type=int,
        default=None,
        help='Número máximo de e-mails a processar (padrão: todos)'
    )

    parser.add_argument(
        '--days_back',
        type=int,
        default=1,
        help='Número de dias para trás a partir da data atual para buscar e-mails (padrão: todos)'
    )

    parser.add_argument(
        '--has_attachments',
        choices=['True', 'False'],
        default=True,
        help='True: apenas com anexos. False: apenas sem anexos. Omitido sem filtro (padrão: nenhum filtro)'
    )

    parser.add_argument(
        '--download_dir',
        type=str,
        default="attachments",
        help='Diretório onde os anexos serão salvos (padrão: diretório atual)'
    )

    parser.add_argument(
        '--from_email',
        type=str,
        default=None,
        help='E-mail do remetente (padrão: nenhum filtro)'
    )

    args = parser.parse_args()
    print(f"Argumentos recebidos: {args}")
    # Obtém serviço autenticado
    service = get_gmail_service()
    print(f"Serviço do Gmail obtido: {'Sim' if service else 'Não'}")
    if service is None:
        print("Não foi possível autenticar o serviço do Gmail. Verifique as credenciais e tente novamente.")
        return  # encerra se não conseguiu autenticar

    # Busca os IDs com os filtros
    print("\n🔍 Buscando IDs de e-mails com os filtros aplicados...")
    print(f"args.has_attachments = {args.has_attachments}")
    ids = fetch_email_ids(
        service,
        days_back=args.days_back,
        max_results=args.max_results,
        has_attachments=args.has_attachments,
        from_email=args.from_email  
    )

    print(f"args.has_attachments = {args.has_attachments}")

    print(f"\n Total de IDs encontrados: {len(ids)}")

    if ids:
        # Exemplo: salva os IDs processados
        save_processed_ids(ids)
        # Você pode adicionar aqui o processamento desejado
    else:
        print("Nenhum ID encontrado com os filtros atuais.")

    if ids and args.download_dir is not None:
        print("\n📥 Baixando anexos...")
        all_downloaded = []
        
        for i, msg_id in enumerate(ids, 1):
            print(f"\nProcessando e-mail {i}/{len(ids)} (ID: {msg_id})")
            downloaded = download_attachments(
                service, 
                msg_id, 
                download_dir=args.download_dir
            )
            all_downloaded.extend(downloaded)
            
            # Pequena pausa para não sobrecarregar a API
            if i < len(ids):
                time.sleep(0.5)
        
        print(f"\n Total de anexos baixados: {len(all_downloaded)}")
    
    # Salva os IDs processados (opcional)
    save_processed_ids(ids)

In [36]:
main()

Argumentos recebidos: Namespace(max_results=None, days_back=1, has_attachments=True, download_dir='attachments', from_email='/run/user/1000/jupyter/runtime/kernel-v30c73aa0e21e129243c6101334a7073d2a7ae29cc.json')
Autenticação bem-sucedida!
Serviço do Gmail obtido: Sim

🔍 Buscando IDs de e-mails com os filtros aplicados...
args.has_attachments = True

          Filtros aplicados: 
          - days_back=1 dias, 
          - max_results=None, 
          - has_attachments=True, 
          - from_email=/run/user/1000/jupyter/runtime/kernel-v30c73aa0e21e129243c6101334a7073d2a7ae29cc.json
          ------------------------
Nenhum e-mail encontrado com os filtros aplicados.
args.has_attachments = True

 Total de IDs encontrados: 0
Nenhum ID encontrado com os filtros atuais.
IDs salvos em /home/priscila/Documentos/estudos/portfolio/data/processed_ids.json


#